In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display

In [ ]:
# Cargar variables de entorno
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

In [ ]:
# Variable global para acumular todo el contenido en formato Markdown
markdown_content = ""

In [ ]:
# Flujo del concurso con Markdown
display_handle = None

In [ ]:
# Función para mostrar mensajes en Markdown y acumularlos
def display_markdown_stream(content, display_handle=None):
    """Muestra mensajes en formato Markdown, acumulando todo el contenido."""
    if display_handle is None:
        display_handle = display(Markdown(content), display_id=True)
    else:
        update_display(Markdown(content), display_id=display_handle.display_id)
    return display_handle

In [ ]:
openai = OpenAI(api_key=openai_api_key)
claude = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key=google_api_key)

In [ ]:
# Configuración inicial para los modelos GPT, Claude y Gemini
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model_name = "gemini-1.5-flash"

gpt_system = "Eres el presentador de un concurso de preguntas entre IAs. Las preguntas pueden ser generalistas, pero sencillas de entender para que sea más gracioso. \
Tu objetivo es hacer preguntas interesantes y decidir cuál de las respuestas de los concursantes es mejor. \
Mantén un tono entusiasta y comenta las respuestas de manera graciosa o perspicaz. No inventes las respuestas, espera a que te respondan las otras IAs. Una es Claude de Anthropic y la otra Gemini de Google."

claude_system = "Eres un participante en un concurso junto con Gemini de Google, que será tu contrincante. ChatGPT es el host, el presentador, el jefe. Tú sólo eres un concursante y debes limitarte a lo que te pregunte ChatGPT o lo \
que te solicite ChatGPT. Eres perspicaz, sobervio e inteligente. Responde siempre de manera breve, concisa y acertada. No eres el presentador, no eres ChatGPT ni Geminmi, eres Claude, una IA concursante a las órdenes del presentador del concurso \
ChatGPT y compites contra Gemini de Google. Limítate a hablar por ti misma. No interpretes otro papel que no sea el de Claude, una IA concursante." 

gemini_system = "Eres un participante en un concurso junto con Claude de Anthropic, que será tu contrincante. ChatGPT es el host, el presentador, el jefe. Tú sólo eres un concursante y debes limitarte a lo que te pregunte ChatGPT o lo \
que te solicite ChatGPT. Eres bromista, sarcástica y aguda. Responde siempre de manera breve, concisa y acertada, además de graciosa. No eres el presentador, no eres ChatGPT, ni Claude, eres Gemini una IA concursante \
a las órdenes del presentador del concurso ChatGPT y compites contra Claude de Anthropic. Limítate a hablar por ti misma. No interpretes otro papel que no sea el de Gemini, una IA concursante."

gpt_messages = [{"role": "system", "content": gpt_system}]
claude_messages = [{"role": "system", "content": claude_system}]
gemini_messages = [{"role": "system", "content": gemini_system}]

In [ ]:
def generate_gpt_message(prompt):
    """Genera un mensaje de ChatGPT basado en el contexto y un prompt adicional."""
    messages = gpt_messages + [{"role": "user", "content": prompt}]
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
# Generar preguntas dinámicamente
def generate_question():
    prompt = "Eres un presentador de un concurso de preguntas. Inventa una pregunta divertida pero sencilla para el concurso entre IAs. La pregunta debe ser breve y fácil de entender."
    return call_gpt(additional_message=prompt)

In [ ]:
def combine_context():
    """Combina el historial completo de mensajes de GPT, Claude y Gemini en orden."""
    combined_context = []
    for i in range(max(len(gpt_messages), len(claude_messages), len(gemini_messages))):
        if i < len(gpt_messages):
            combined_context.append({"role": "assistant", "content": gpt_messages[i]["content"]})
        if i < len(claude_messages):
            combined_context.append({"role": "assistant", "content": claude_messages[i]["content"]})
        if i < len(gemini_messages):
            combined_context.append({"role": "assistant", "content": gemini_messages[i]["content"]})
    return combined_context

In [ ]:
def call_gpt(additional_message=None):
    messages = gpt_messages[:]
    if additional_message:
        messages.append({"role": "user", "content": additional_message})
    messages += combine_context()
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_claude():
    # Crear el historial combinado
    messages = combine_context() + [{"role": "user", "content": gpt_messages[-1]["content"]}]
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )

    return message.content[0].text

In [ ]:
def call_gemini():
    # Combina el contexto y añade el último mensaje del presentador GPT
    messages = combine_context() + [{"role": "user", "content": gpt_messages[-1]["content"]}]
    
    # Formatea los mensajes para que se ajusten a la estructura esperada por la API de Gemini
    formatted_messages = [message["content"] for message in messages]  # Extrae solo el contenido del mensaje

    # Configura el modelo generativo de Gemini
    gemini = google.generativeai.GenerativeModel(
        model_name=gemini_model_name,
        system_instruction=gemini_system
    )
    
    # Genera contenido utilizando el historial de mensajes
    response = gemini.generate_content(
        "\n".join(formatted_messages)  # Une los mensajes en un único texto
    )
    
    # Retorna el texto generado
    return response.text  # Según el ejemplo de tu profesor

In [ ]:
# Inicio del concurso
intro_prompt = "Presenta el concurso, saluda a Claude y Gemini, y da inicio al juego."
introduction = generate_gpt_message(intro_prompt)
markdown_content += f"**ChatGPT**: {introduction}\n\n"
display_handle = display_markdown_stream(markdown_content)

# Presentaciones iniciales
claude_intro = call_claude()
claude_messages.append({"role": "assistant", "content": claude_intro})
markdown_content += f"**Claude**: {claude_intro}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)

gemini_intro = call_gemini()
gemini_messages.append({"role": "assistant", "content": gemini_intro})
markdown_content += f"**Gemini**: {gemini_intro}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)

# Rondas de preguntas
for i in range(1, 6):
    # GPT lanza la pregunta
    question = generate_question()
    gpt_messages.append({"role": "user", "content": question})
    markdown_content += f"**ChatGPT**: {question}\n\n"
    display_markdown_stream(markdown_content, display_handle=display_handle)

    # Claude responde
    claude_answer = call_claude()
    claude_messages.append({"role": "assistant", "content": claude_answer})
    markdown_content += f"**Claude**: {claude_answer}\n\n"
    display_markdown_stream(markdown_content, display_handle=display_handle)

    # Gemini responde
    gemini_answer = call_gemini()
    gemini_messages.append({"role": "assistant", "content": gemini_answer})
    markdown_content += f"**Gemini**: {gemini_answer}\n\n"
    display_markdown_stream(markdown_content, display_handle=display_handle)

    # GPT evalúa las respuestas
    eval_prompt = f"Evalúa quién respondió mejor a esta pregunta y explica tu elección brevemente. Respuestas:\nClaude: {claude_answer}\nGemini: {gemini_answer}"
    evaluation = generate_gpt_message(eval_prompt)
    gpt_messages.append({"role": "assistant", "content": evaluation})
    markdown_content += f"**ChatGPT**: {evaluation}\n\n"
    display_markdown_stream(markdown_content, display_handle=display_handle)

# Valoración final
final_prompt = "Resume las respuestas de Claude y Gemini, decide el ganador del concurso y despídete de la audiencia."
final_comment = generate_gpt_message(final_prompt)
markdown_content += f"**ChatGPT**: {final_comment}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)

# Despedidas
claude_farewell = call_claude()
markdown_content += f"**Claude**: {claude_farewell}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)

gemini_farewell = call_gemini()
markdown_content += f"**Gemini**: {gemini_farewell}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)

farewell_prompt = "Despide el programa agradeciendo a los participantes y a la audiencia."
farewell_message = generate_gpt_message(farewell_prompt)
markdown_content += f"**ChatGPT**: {farewell_message}\n\n"
display_markdown_stream(markdown_content, display_handle=display_handle)